In [35]:
import numpy as np
with open("input_13") as f:
    input = f.read().split("\n\n")

In [2]:
machines = []
for line in input:
    split = line.split("\n")
    machine = {}
    machine["A"] = np.asarray((int(split[0][12:14]), int(split[0][-2:])) )
    machine["B"] = np.asarray((int(split[1][12:14]), int(split[1][-2:])) )
    prize = split[2].split(" ")
    machine["prize"] = np.asarray((int(prize[1][2:-1]), int(prize[2][2:])) )
    machines.append(machine)

## Part 1:

In [3]:
def find_lowest_price(machine):
    # start from low A and high B
    avals = np.arange(101)
    bvals = np.arange(101,0,-1)
    for a in avals:
        for b in bvals:
            if np.all(a * machine["A"] + b * machine["B"] == machine["prize"]):
                return (a*3 + b).item()
    return 0

In [ ]:
c = 0
for machine in machines:
    c += find_lowest_price(machine)
c

In [5]:
# works too, but faster
def find_combination(A, B, sum):
    # want A small, B large
    # starting values:
    num_B = np.max(sum // B)
    delta = sum - num_B * B
    if np.all(delta == 0):
        return(0, num_B)
    while num_B > 0:
        num_B -= 1
        delta += B
        num_A = delta // A
        if np.all(delta % A == 0) and num_A[0] == num_A[1]:
            return (num_A[0], num_B)
    return None

In [ ]:
c = 0
for machine in machines:
    combination = find_combination(machine["A"], machine["B"], machine["prize"])
    if combination is not None:
        A,B = combination
        c += A * 3 + B
c.item()

## Part 2:

In [ ]:
number_to_add = 10000000000000
for machine in machines:
    machine["prize"] += number_to_add

We want to find solution to num_a * a + num_b * b = sum (1) \
A solution exists iff gcd(a,b)|c. \
First find solution for separate dimensions:

In [166]:
def find_num_A(A, B, sum):
    #check if solution exists:
    if sum % math.gcd(A, B) != 0:
        return None
    # fill sum with B, then subtract until n*a fit
    # starting values:
    delta = sum % B
    if delta == 0:
        return 0
    while delta < sum:
        delta += B
        if delta % A == 0:
            return delta // A

Now we have a solution given by num_a0, num_b0. The general solution is: \
num_a = num_a0 - r * b/gcd(a,b) and num_b = num_b0 + r * a/gcd(a,b) with r any integer. \
After doing this for both dimensions, we require: num_a_x = num_a_y and num_b_x = num_b_y. \
Then you solve for r_x and r_y and plug it into the formulas for num_a and num_b.

In [169]:
def find_combination_2D(machine):
    A = machine["A"]
    B = machine["B"]
    sum = machine["prize"]
    gcd = [math.gcd(a,b) for a,b in zip(A,B)]
    num_ax = find_num_A(A[0], B[0], sum[0])
    num_ay = find_num_A(A[1], B[1], sum[1])
    if None in (num_ax, num_ay):
        return 0,0
    num_bx = (sum[0] - num_ax*A[0]) // B[0]
    num_by = (sum[1] - num_ay*A[1]) // B[1]
    r = (gcd[0] * (A[1] * (num_ax - num_ay) + B[1] * (num_bx - num_by))) / (B[0]*A[1] - B[1]*A[0])
    # need to check if r ist int:
    if not r.is_integer():
        return 0,0
    num_a = num_ax - int(r) * B[0] // gcd[0]
    num_b = num_bx + int(r) * A[0] // gcd[0]
    return np.array((num_a, num_b))

In [ ]:
c = 0
weights = np.array((3,1))
for machine in machines:
    c += np.dot(weights, find_combination_2D(machine))
c.item()